In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import re

In [3]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [4]:
driver.get('http://www.linkedin.com')

In [5]:
email_id = input('Enter your email-id ')
password = input("Enter your password ")

Enter your email-id pundhird@gmail.com
Enter your password shiv@1468


In [6]:
driver.find_element_by_id("login-email").send_keys(email_id)
driver.find_element_by_id("login-password").send_keys(password)
driver.find_element_by_id("login-submit").click()

In [7]:
company_name = []
company_linkedin_url = []
employee_fname = []
employee_lname = []
employee_url = []
designation = []
location = []

In [8]:
company_list = pd.read_csv("company.csv")
designation_list = pd.read_csv("designation.csv")

In [9]:
def func(durl,comp_no):
    k=1
    time.sleep(3)
    while(True):
        url_p = durl+'&page='+str(k)
        driver.get(url_p)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html)
        l1 = driver.find_elements_by_class_name('search-result__wrapper')
        aa = soup.findAll('a')
        for i in range(len(aa)/2):
            if(re.match(r'^/in/', aa[i*2]['href'])):
                driver.get('http://www.linkedin.com'+aa[i]['href'])
                time.sleep(2)
                employee_fname.append(driver.find_element_by_class_name('pv-top-card-section__name').text.split(" ")[0])
                employee_lname.append(driver.find_element_by_class_name('pv-top-card-section__name').text.split(" ")[1:])
                company_name.append(company_list.Company[comp_no])
                company_linkedin_url.append(company_list['Linkedin company URL'][comp_no])
                employee_url.append(driver.current_url)
                designation.append(driver.find_element_by_class_name('pv-top-card-section__headline').text)
                location.append(driver.find_element_by_class_name('pv-top-card-section__location').text)
                
                
        if(len(l1)==0):
            break
        
        k+=1

In [ ]:
import time
for i in range(0,len(company_list)):
    driver.get(company_list['Linkedin company URL'][i])
    time.sleep(2)
    try:
        driver.find_element_by_partial_link_text("See all").click()
        time.sleep(2)
    
    except NoSuchElementException as e:
        pass
    
    except WebDriverException:
        driver.find_element_by_class_name("msg-overlay-bubble-header").click()
        driver.find_element_by_partial_link_text("See all").click()
        time.sleep(2)
        
    finally:
        func(driver.current_url,i)

In [43]:
df = pd.DataFrame({'Emp_fname':employee_fname,
                 'Emp_lname':employee_lname,
                 'Emp_URL':employee_url,
                 'Designation':designation,
                 'Company':company_name,
                 'Company_URL':company_linkedin_url,
                 'Location':location})

In [49]:
df.to_excel('linkedin.xlsx',index=False)